In [10]:
import pandas as pd
import joblib

In [20]:
from src.data_fetcher import get_fpl_data

data = get_fpl_data()
data


(     can_transact  can_select  chance_of_playing_next_round  \
 0            True       False                           0.0   
 1            True       False                           0.0   
 2            True       False                           0.0   
 3            True       False                         100.0   
 4            True       False                           0.0   
 ..            ...         ...                           ...   
 799          True       False                         100.0   
 800          True       False                           NaN   
 801          True       False                           NaN   
 802          True       False                         100.0   
 803          True       False                           NaN   
 
      chance_of_playing_this_round       code  cost_change_event  \
 0                             0.0     438098                  0   
 1                             0.0     205651                  0   
 2                        

In [13]:
import os
print(os.path.exists("/Users/joeriksson/fehrman-git/fpl_bot/models/fpl_ai_model.pkl"))  # True if correct

True


In [14]:
model = joblib.load("/Users/joeriksson/fehrman-git/fpl_bot/models/fpl_ai_model.pkl")

/Users/joeriksson/fehrman-git/fpl_bot/.venv/lib/python3.10/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Ridge from version 1.1.3 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
# 3. Simulate a few rows of sample FPL API data
players = pd.DataFrame({
    "first_name": ["Erling", "Bukayo", "Alisson"],
    "second_name": ["Haaland", "Saka", "Becker"],
    "now_cost": [120, 85, 55],  # in tenths of millions
    "minutes": [3000, 2900, 3200],
    "form": [9.2, 8.1, 5.5],
    "transfers_in": [400000, 350000, 200000],
    "goals_scored": [30, 14, 0],
    "assists": [5, 11, 0],
    "yellow_cards": [3, 2, 1],
    "threat": [80.5, 65.0, 10.0],
    "influence": [90.0, 72.5, 50.0]
})

# 4. Define features
FEATURE_COLUMNS = [
    "now_cost", "minutes", "form", "transfers_in",
    "goals_scored", "assists", "yellow_cards", "threat", "influence"
]

# 5. Predict
X = players[FEATURE_COLUMNS]
players["expected_points"] = model.predict(X)

# 6. Show results
players[["first_name", "second_name", "expected_points"]]

,first_name,second_name,expected_points
0,Erling,Haaland,89.425396
1,Bukayo,Saka,111.263998
2,Alisson,Becker,145.517962


In [16]:
players

,first_name,second_name,now_cost,minutes,form,transfers_in,goals_scored,assists,yellow_cards,threat,influence,expected_points
0,Erling,Haaland,120,3000,9.2,400000,30,5,3,80.5,90.0,89.425396
1,Bukayo,Saka,85,2900,8.1,350000,14,11,2,65.0,72.5,111.263998
2,Alisson,Becker,55,3200,5.5,200000,0,0,1,10.0,50.0,145.517962
